<a href="https://colab.research.google.com/github/DAnand05/Animation/blob/main/Trash_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!ls /content/drive/MyDrive/trashnet.zip


/content/drive/MyDrive/trashnet.zip


In [3]:
import zipfile
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/trashnet.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense

In [6]:
# Data directories
train_dir = "/content/data/train"
val_dir = "/content/data/test"
test_dir = "/content/data/test"  # Optional for final evaluation

# Image size for preprocessing
img_width, img_height = 224, 224

# Data augmentation configuration
datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,
    fill_mode="nearest"
)

# Train data generator with augmentation
train_datagen = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=32,  # Adjust batch size based on GPU memory
    class_mode="categorical"  # Multi-class classification
)

# Validation data generator (no augmentation)
val_datagen = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode="categorical"
)

# Pre-trained VGG16 model (include weights='imagenet')
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(img_width, img_height, 3))
base_model.trainable = False  # Freeze base model layers

# Add custom classification layers
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)  # Experiment with different neuron numbers
predictions = Dense(6, activation="softmax")(x)  # Output layer for 6 trash classes

# Create the final model
model = keras.Model(inputs=base_model.input, outputs=predictions)

# Model compilation (adjust learning rate, optimizer based on hyperparameter tuning)
model.compile(loss="categorical_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              metrics=["accuracy"])


Found 2019 images belonging to 6 classes.
Found 508 images belonging to 6 classes.


In [ ]:
# Model training
history = model.fit(
    train_datagen,
    steps_per_epoch=len(train_datagen),  # Adjust based on training data size
    epochs=20,  # Adjust epochs based on validation results
    validation_data=val_datagen,
    validation_steps=len(val_datagen)  # Adjust based on validation data size
)

Epoch 1/20
17/64 [======>.......................] - ETA: 15:20 - loss: 1.7015 - accuracy: 0.3825

In [ ]:
# Model evaluation on test set (optional)
if test_dir:
    test_datagen = datagen.flow_from_directory(
        test_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode="categorical"
    )
    test_loss, test_acc = model.evaluate(test_datagen)
    print("Test accuracy:", test_acc)

# Save the model for future use (optional)
model.save("trash_classifier.h5")

# Hyperparameter tuning (optional)
# You can use techniques like grid search or random search to find optimal hyperparameters.
# Here's a basic example with grid search:

from sklearn.model_selection import GridSearchCV

param_grid = {
    "learning_rate": [0.0001, 0.001, 0.01],
    "optimizer": [keras.optimizers.Adam, keras.optimizers.SGD],
    "epochs": [10, 20, 30]
}

grid_search = GridSearchCV(model, param_grid, cv=3)  # Adjust cv (cross-validation) folds
grid_search.fit(train_datagen)

best_model = grid_search.best_estimator_
print("Best hyperparameters:", grid_search.best_params_)

# Evaluate the best model on test set (optional)
if test_dir:
    test_loss, test_acc = best_model.evaluate(test_datagen)
    print("Test accuracy with best hyperparameters:", test_acc)

# Further considerations
* Explore other CNN architectures like ResNet or EfficientNet for potentially better performance.
* Consider class imbalance handling techniques (oversampling, undersampling) if applicable.
* Visualize training curves (loss, accuracy) to monitor model learning progress.
* Experiment with different data augmentation techniques and parameters.
* Use techniques like early stopping to prevent overfitting.